# Praca domowa 2
# 20.12.2020 r.

# Jakub Zbrzezny
# Nr indeksu: 286689
# Nick do publikowania wyników: Heihachi46

## Przygotowanie danych

In [590]:
import pandas as pd
import numpy as np

In [591]:
Badges = pd.read_csv("Badges.csv")
Comments = pd.read_csv("Comments.csv")
PostLinks = pd.read_csv("PostLinks.csv")
Posts = pd.read_csv("Posts.csv")
Tags = pd.read_csv("Tags.csv")
Users = pd.read_csv("Users.csv")
Votes = pd.read_csv("Votes.csv")

In [592]:
import os, os.path
import sqlite3
import tempfile

# Ścieżka dostępu do bazy danych:
baza = os.path.join(tempfile.mkdtemp(), 'Moja_baza.db')
if os.path.isfile(baza): # Jeśli baza już istnieje...
    os.remove(baza)      # ...usunę ją.
    
conn = sqlite3.connect(baza)      # Połączenie do bazy danych.

Badges.to_sql("Badges", conn)     # Importuję ramkę danych do bazy danych.
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)
Posts.to_sql("Posts", conn)
Tags.to_sql("Tags", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

## Rozwiązania zadań

### Zadanie 1

#### a) Zapytanie SQL

In [593]:
wyn_sql_1 = pd.read_sql_query("""
SELECT Posts.Title, RelatedTab.NumLinks
FROM
    (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
     FROM PostLinks
     GROUP BY RelatedPostId) AS RelatedTab
JOIN Posts ON RelatedTab.PostId = Posts.Id
WHERE Posts.PostTypeId = 1
ORDER BY NumLinks DESC
""", conn)

In [594]:
wyn_sql_1.head(20)

,Title,NumLinks
0,Is there a way to find out if I need a transit...,594
1,Do I need a visa to transit (or layover) in th...,585
2,Should my first trip be to the country which i...,331
3,Should I submit bank statements when applying ...,259
4,How much electronics and other valuables can I...,197
5,How does the Schengen 90/180 rule work?,192
6,Do I need a US visa to transit (or layover) th...,165
7,I have two passports/nationalities. How do I u...,146
8,Schengen Visa Refusal: Justification for the p...,111
9,Can I visit Schengen countries on a Type-D Sch...,96


#### b) Rozwiązanie z pakietu "pandas"

In [595]:
RelatedTab = pd.DataFrame(PostLinks.groupby("RelatedPostId").apply(len).reset_index().rename(columns = {"RelatedPostId":"PostId", 0:"NumLinks"}))

RelatedTab.head(10)

,PostId,NumLinks
0,1,1
1,2,3
2,6,5
3,9,1
4,10,94
5,11,6
6,16,10
7,25,18
8,26,5
9,28,1


#### Interpretacja zapytania dla ramki RelatedTab

W ramce RelatedTab, wyświetlamy, ile każdy post ma numlinków.

#### Ciąg dalszy rozwiązania

In [596]:
zlacz_RelTab_Posts = pd.merge(Posts, RelatedTab, left_on = "Id", right_on = "PostId", how = "inner")
zlacz_RelTab_Posts = zlacz_RelTab_Posts.loc[zlacz_RelTab_Posts.PostTypeId == 1]

# Sortuję malejąco.
zlacz_RelTab_Posts = zlacz_RelTab_Posts.sort_values(by = "NumLinks", ascending = False)
wyn_pandas_1 = zlacz_RelTab_Posts.loc[:, ["Title", "NumLinks"]].drop_duplicates().reset_index(drop = True)

In [597]:
wyn_pandas_1.head(20)

,Title,NumLinks
0,Is there a way to find out if I need a transit...,594
1,Do I need a visa to transit (or layover) in th...,585
2,Should my first trip be to the country which i...,331
3,Should I submit bank statements when applying ...,259
4,How much electronics and other valuables can I...,197
5,How does the Schengen 90/180 rule work?,192
6,Do I need a US visa to transit (or layover) th...,165
7,I have two passports/nationalities. How do I u...,146
8,Schengen Visa Refusal: Justification for the p...,111
9,Can I visit Schengen countries on a Type-D Sch...,96


#### Interpretacja zapytania dla ramki wynikowej

W otrzymanej ramce, w k-tym wierszu, wyświetlamy, jaki jest tytuł dla k-tej największej liczby numlinków. Rozpatrujemy tytuły, gdzie identyfikator typu postu jest równy 1.

#### Porównanie wyników

In [598]:
wyn_sql_1.equals(wyn_pandas_1)

False

Mimo, że piersze 10 wierszy z zapytania SQL i ramki otrzymanej za pomocą "pandas" się zgadzają, to wynik jest False. Prawdopodobnie kolejność jest inna (możliwe, że dla wierszy o takiej samej wartości NumLinks, tytuły są ustawione w innej kolejności). Funkcja .equals() z pakietu "pandas" wymaga niestety, by kolejność była taka sama. Poniżej jest przykład, który pokazuje, że kolejność musi być jednakowa. Porównam ramki danych więc inną metodą (która nie wymaga takiej samej kolejności).

In [599]:
a = pd.DataFrame([2, 3, 4])
a

,0
0,2
1,3
2,4


In [600]:
b = pd.DataFrame([4, 2, 3])
b

,0
0,4
1,2
2,3


In [601]:
a.equals(b)

False

Powyższe oba zbiory zawierają takie same elementy: 2, 3, 4, ale kolejności są różne. Zwracany wynik to: "False".

In [602]:
wyn_sql_1.loc[:, "NumLinks"].equals(wyn_pandas_1.loc[:, "NumLinks"])

True

Rzeczywiście, wartości NumLinks są dobrze uporządkowane.

In [603]:
wyn_sql_1.loc[:, "Title"].equals(wyn_pandas_1.loc[:, "Title"])

False

Ale tytuły są już w innej kolejności.

Zatem sprawdzę tożsamość wyników w następujący sposób.
Obie ramki posortuję po tytule rosnąco i po numlinkach malejąco i wtedy porównam je funkcją .equals() z pakietu "pandas".

In [604]:
wyn_sql_1.sort_values(by = ["Title", "NumLinks"], ascending = [True, False]).reset_index(drop = True).equals(wyn_pandas_1.sort_values(by = ["Title", "NumLinks"], ascending = [True, False]).reset_index(drop = True))

True

Stąd oba wyniki są tożsame.

#### Porównanie czasów obu rozwiązań.

In [605]:
import timeit as ti

##### a) Zapytanie SQL

In [606]:
rozw_sql_1 = '''
def fun_sql_1():
    wyn_sql_1 = pd.read_sql_query("""
    SELECT Posts.Title, RelatedTab.NumLinks
    FROM
        (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
         FROM PostLinks
         GROUP BY RelatedPostId) AS RelatedTab
    JOIN Posts ON RelatedTab.PostId = Posts.Id
    WHERE Posts.PostTypeId = 1
    ORDER BY NumLinks DESC
    """, conn)
'''

In [607]:
time_sql_1 = ti.timeit(stmt = rozw_sql_1)
print(time_sql_1)

0.07000429999970947


In [608]:
time_sql_1 = ti.timeit(stmt = rozw_sql_1)
print(time_sql_1)

0.08675529999891296


In [609]:
time_sql_1 = ti.timeit(stmt = rozw_sql_1)
print(time_sql_1)

0.08641229999921052


Widać więc, że czasy wyraźnie oscylują. Wezmę więc średnią dla stu iteracji.

In [610]:
time_sql_1 = 0
for i in range(100):
    time_sql_1 = time_sql_1 + ti.timeit(stmt = rozw_sql_1)

time_sql_1 = time_sql_1 / 100

time_sql_1

0.06543988399982481

In [611]:
f"Czyli średni czas utworzenia ramki danych za pomocą zapytania SQL, to {time_sql_1} sekund."

'Czyli średni czas utworzenia ramki danych za pomocą zapytania SQL, to 0.06543988399982481 sekund.'

##### b) Rozwiązanie z pakietu "pandas"

In [612]:
rozw_pandas_1 = '''
def fun_pandas_1():
    RelatedTab = pd.DataFrame(PostLinks.groupby("RelatedPostId").apply(len).reset_index().rename(columns = {"RelatedPostId":"PostId", 0:"NumLinks"}))
    zlacz_RelTab_Posts = pd.merge(Posts, RelatedTab, left_on = "Id", right_on = "PostId", how = "inner")
    zlacz_RelTab_Posts = zlacz_RelTab_Posts.loc[zlacz_RelTab_Posts.PostTypeId == 1]
    # Sortuję malejąco.
    zlacz_RelTab_Posts = zlacz_RelTab_Posts.sort_values(by = "NumLinks", ascending = False)
    wyn_pandas_1 = zlacz_RelTab_Posts.loc[:, ["Title", "NumLinks"]].drop_duplicates().reset_index(drop = True)
'''

In [613]:
time_pandas_1 = 0
for i in range(100):
    time_pandas_1 = time_pandas_1 + ti.timeit(stmt = rozw_pandas_1)

time_pandas_1 = time_pandas_1 / 100

time_pandas_1

0.07300254399993719

In [614]:
f"Czyli średni czas utworzenia ramki danych za pomocą pakietu 'pandas', to {time_pandas_1} sekund."

"Czyli średni czas utworzenia ramki danych za pomocą pakietu 'pandas', to 0.07300254399993719 sekund."

In [615]:
time_pandas_1 / time_sql_1

1.1155665251505127

Zatem czas utworzenia ramki danych za pomocą pakietu "pandas" jest zbliżony do czasu utworzenia ramki danych za pomocą zapytania SQL.

### Zadanie 2

#### a) Zapytanie SQL

In [616]:
wyn_sql_2 = pd.read_sql_query("""
SELECT
    Users.DisplayName,
    Users.Age,
    Users.Location,
    SUM(Posts.FavoriteCount) AS FavoriteTotal,
    Posts.Title AS MostFavoriteQuestion,
    MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id = Posts.OwnerUserId
WHERE Posts.PostTypeId = 1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
""", conn)

In [617]:
wyn_sql_2

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,None,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,None,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,None,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


#### b) Rozwiązanie z pakietu "pandas"

In [618]:
zlacz_Posts_Users = pd.DataFrame(pd.merge(Users, Posts, left_on = "Id", right_on = "OwnerUserId", how = "inner"))

In [619]:
zlacz_Posts_Users = zlacz_Posts_Users.loc[zlacz_Posts_Users.PostTypeId == 1]

In [620]:
zlacz_Posts_Users = zlacz_Posts_Users.rename(columns = {"Title":"MostFavoriteQuestion"})

In [621]:
grupuj = zlacz_Posts_Users.groupby("OwnerUserId")["FavoriteCount"].agg([sum, max]).reset_index().rename(columns = {"sum":"FavoriteTotal", "max":"MostFavoriteQuestionLikes"})
grupuj.head(10)

,OwnerUserId,FavoriteTotal,MostFavoriteQuestionLikes
0,8.0,13.0,13.0
1,9.0,0.0,NaN
2,10.0,40.0,35.0
3,11.0,17.0,4.0
4,13.0,58.0,49.0
5,16.0,3.0,3.0
6,19.0,109.0,34.0
7,22.0,0.0,NaN
8,23.0,14.0,8.0
9,24.0,0.0,NaN


In [622]:
dalsze_zlaczenie = pd.merge(zlacz_Posts_Users, grupuj, how = "inner", left_on = ["OwnerUserId", "FavoriteCount"], 
right_on = ["OwnerUserId", "MostFavoriteQuestionLikes"])

In [623]:
dalsze_zlaczenie = dalsze_zlaczenie.sort_values(by = "FavoriteTotal", ascending = False)

In [624]:
wyn_pandas_2 = dalsze_zlaczenie.loc[:, ["DisplayName", "Age", "Location", "FavoriteTotal", "MostFavoriteQuestion", "MostFavoriteQuestionLikes"]].head(10).reset_index(drop = True)

In [625]:
wyn_pandas_2

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,NaN,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,NaN,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,NaN,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


#### Interpretacja zapytania dla ramki wynikowej

W otrzymanej ramce danych wyświetlamy, wyświetlaną nazwę użytkownika, jego wiek, lokalizację, całkowitą ilość ulubionych postów, ulubione pytanie, największą ilość ulubionych pytań dla każdego ID właściciela postu.

#### Porównanie wyników

In [626]:
wyn_sql_2.equals(wyn_pandas_2)

True

Stąd oba wyniki są tożsame.

#### Porównanie czasów obu rozwiązań.

##### a) Zapytanie SQL

In [627]:
rozw_sql_2 = '''
def fun_sql_2():
    wyn_sql_2 = pd.read_sql_query("""
    SELECT
        Users.DisplayName,
        Users.Age,
        Users.Location,
        SUM(Posts.FavoriteCount) AS FavoriteTotal,
        Posts.Title AS MostFavoriteQuestion,
        MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
    FROM Posts
    JOIN Users ON Users.Id = Posts.OwnerUserId
    WHERE Posts.PostTypeId = 1
    GROUP BY OwnerUserId
    ORDER BY FavoriteTotal DESC
    LIMIT 10
    """, conn)
'''

In [628]:
time_sql_2 = 0
for i in range(100):
    time_sql_2 = time_sql_2 + ti.timeit(stmt = rozw_sql_2)

time_sql_2 = time_sql_2 / 100

time_sql_2

0.07997642000002088

In [629]:
f"Czyli średni czas utworzenia ramki danych za pomocą zapytania SQL, to {time_sql_2} sekund."

'Czyli średni czas utworzenia ramki danych za pomocą zapytania SQL, to 0.07997642000002088 sekund.'

##### b) Rozwiązanie z pakietu "pandas"

In [630]:
rozw_pandas_2 = '''
def fun_pandas_2():
    zlacz_Posts_Users = pd.DataFrame(pd.merge(Users, Posts, left_on = "Id", right_on = "OwnerUserId", how = "inner"))
    zlacz_Posts_Users = zlacz_Posts_Users.loc[zlacz_Posts_Users.PostTypeId == 1]
    zlacz_Posts_Users = zlacz_Posts_Users.rename(columns = {"Title":"MostFavoriteQuestion"})
    grupuj = zlacz_Posts_Users.groupby("OwnerUserId")["FavoriteCount"].agg([sum, max]).reset_index().rename(columns = {"sum":"FavoriteTotal", "max":"MostFavoriteQuestionLikes"})
    dalsze_zlaczenie = pd.merge(zlacz_Posts_Users, grupuj, how = "inner", left_on = ["OwnerUserId", "FavoriteCount"], 
    right_on = ["OwnerUserId", "MostFavoriteQuestionLikes"])
    dalsze_zlaczenie = dalsze_zlaczenie.sort_values(by = "FavoriteTotal", ascending = False)
    wyn_pandas_2 = dalsze_zlaczenie.loc[:, ["DisplayName", "Age", "Location", "FavoriteTotal", "MostFavoriteQuestion", "MostFavoriteQuestionLikes"]].head(10).reset_index(drop = True)
'''

In [631]:
time_pandas_2 = 0
for i in range(100):
    time_pandas_2 = time_pandas_2 + ti.timeit(stmt = rozw_pandas_2)

time_pandas_2 = time_pandas_2 / 100

time_pandas_2

0.07948346699988179

In [632]:
f"Czyli średni czas utworzenia ramki danych za pomocą pakietu 'pandas', to {time_pandas_2} sekund."

"Czyli średni czas utworzenia ramki danych za pomocą pakietu 'pandas', to 0.07948346699988179 sekund."

In [633]:
time_pandas_2 / time_sql_2

0.9938362707390633

Zatem czas utworzenia ramki danych za pomocą pakietu "pandas" jest też zbliżony do czasu utworzenia ramki danych za pomocą zapytania SQL.

### Zadanie 3

#### a) Zapytanie SQL

In [634]:
wyn_sql_3 = pd.read_sql_query("""
SELECT
    Posts.Title,
    CmtTotScr.CommentsTotalScore
FROM (
        SELECT
            PostID,
            UserID,
            SUM(Score) AS CommentsTotalScore
        FROM Comments
        GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID = CmtTotScr.PostID AND Posts.OwnerUserId = CmtTotScr.UserID
WHERE Posts.PostTypeId = 1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10
""", conn)

In [635]:
wyn_sql_3

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


#### b) Rozwiązanie z pakietu "pandas"

In [636]:
CmtTotScr = pd.DataFrame(Comments.groupby(["PostId", "UserId"])["Score"].agg(sum).reset_index().rename(columns = {"Score":"CommentsTotalScore"}))

In [637]:
CmtTotScr.head()

,PostId,UserId,CommentsTotalScore
0,1,9.0,0
1,1,12.0,0
2,1,20.0,3
3,1,65.0,2
4,2,95.0,0


#### Interpretacja zapytania dla ramki CmtTotScr

W ramce danych CmtTotScr wyświetlamy, jaki każdy post dla danego ID użytkownika, ma całkowity wynik z komentarzy.

#### Ciąg dalszy rozwiązania

In [638]:
zlacz_Posts_CmtTotScr = pd.merge(Posts, CmtTotScr, left_on = ["Id", "OwnerUserId"], right_on = ["PostId", "UserId"], how = "inner")

In [639]:
zlacz_Posts_CmtTotScr = zlacz_Posts_CmtTotScr.loc[zlacz_Posts_CmtTotScr.PostTypeId == 1]

In [640]:
zlacz_Posts_CmtTotScr = zlacz_Posts_CmtTotScr.sort_values(by = "CommentsTotalScore", ascending = False)

In [641]:
zlacz_Posts_CmtTotScr = zlacz_Posts_CmtTotScr.loc[:, ["Title", "CommentsTotalScore"]]

In [642]:
wyn_pandas_3 = zlacz_Posts_CmtTotScr.head(10).reset_index(drop = True)

In [643]:
wyn_pandas_3

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,Can I have a watermelon in hand luggage?,25
4,How does President Trump's travel ban affect n...,25
5,What's the longest scheduled public bus ride i...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,India just demonetized all Rs 500 & 1000 notes...,20
9,Legalities and safety concerns of visiting pro...,20


#### Interpretacja zapytania dla ramki wynikowej

W otrzymanej ramce wyświetlamy całkowity wynik z komentarzy dla każdego tytułu postu o ID równym 1.

#### Porównanie wyników

In [644]:
wyn_sql_3.equals(wyn_pandas_3)

False

Mimo, że piersze 10 wierszy z zapytania SQL i ramki otrzymanej za pomocą "pandas" się zgadzają, to wynik jest False. Widać, jak dla wartości CommentsTotalScore = 25, z SQLa i pakietu "pandas", że są te same tytuły (tylko inna kolejność jest). Porównam ramki danych więc inną metodą.

In [645]:
wyn_pandas_3.loc[:, "CommentsTotalScore"].equals(wyn_sql_3.loc[:, "CommentsTotalScore"])

True

Zatem wartości CommentsTotalScore są dobrze uporządkowane.

In [646]:
wyn_pandas_3.loc[:, "Title"].equals(wyn_sql_3.loc[:, "Title"])

False

Tylko wartości tytułów są w innej kolejności.

Zatem sprawdzę tożsamość wyników w sposób analogiczny jak w przypadku 1-go zapytania.

In [647]:
wyn_sql_3.sort_values(by = ["Title", "CommentsTotalScore"], ascending = [True, False]).reset_index(drop = True).equals(wyn_pandas_3.sort_values(by = ["Title", "CommentsTotalScore"], ascending = [True, False]).reset_index(drop = True))

True

Stąd oba wyniki są tożsame.

#### Porównanie czasów obu rozwiązań.

##### a) Zapytanie SQL

In [648]:
rozw_sql_3 = '''
def fun_sql_3():
    wyn_sql_3 = pd.read_sql_query("""
    SELECT
        Posts.Title,
        CmtTotScr.CommentsTotalScore
    FROM (
            SELECT
                PostID,
                UserID,
                SUM(Score) AS CommentsTotalScore
            FROM Comments
            GROUP BY PostID, UserID
    ) AS CmtTotScr
    JOIN Posts ON Posts.ID = CmtTotScr.PostID AND Posts.OwnerUserId = CmtTotScr.UserID
    WHERE Posts.PostTypeId = 1
    ORDER BY CmtTotScr.CommentsTotalScore DESC
    LIMIT 10
    """, conn)
'''

In [649]:
time_sql_3 = 0
for i in range(100):
    time_sql_3 = time_sql_3 + ti.timeit(stmt = rozw_sql_3)

time_sql_3 = time_sql_3 / 100

time_sql_3

0.06464780999976938

In [650]:
f"Czyli średni czas utworzenia ramki danych za pomocą zapytania SQL, to {time_sql_3} sekund."

'Czyli średni czas utworzenia ramki danych za pomocą zapytania SQL, to 0.06464780999976938 sekund.'

##### b) Rozwiązanie z pakietu "pandas"

In [651]:
rozw_pandas_3 = '''
def fun_pandas_3():
    CmtTotScr = pd.DataFrame(Comments.groupby(["PostId", "UserId"])["Score"].agg(sum).reset_index().rename(columns = {"Score":"CommentsTotalScore"}))
    zlacz_Posts_CmtTotScr = pd.merge(Posts, CmtTotScr, left_on = ["Id", "OwnerUserId"], right_on = ["PostId", "UserId"], how = "inner")
    zlacz_Posts_CmtTotScr = zlacz_Posts_CmtTotScr.loc[zlacz_Posts_CmtTotScr.PostTypeId == 1]
    zlacz_Posts_CmtTotScr = zlacz_Posts_CmtTotScr.sort_values(by = "CommentsTotalScore", ascending = False)
    zlacz_Posts_CmtTotScr = zlacz_Posts_CmtTotScr.loc[:, ["Title", "CommentsTotalScore"]]
    wyn_pandas_3 = zlacz_Posts_CmtTotScr.head(10).reset_index(drop = True)
'''

In [652]:
time_pandas_3 = 0
for i in range(100):
    time_pandas_3 = time_pandas_3 + ti.timeit(stmt = rozw_pandas_3)

time_pandas_3 = time_pandas_3 / 100

time_pandas_3

0.06797477100019023

In [653]:
f"Czyli średni czas utworzenia ramki danych za pomocą pakietu 'pandas', to {time_pandas_3} sekund."

"Czyli średni czas utworzenia ramki danych za pomocą pakietu 'pandas', to 0.06797477100019023 sekund."

In [654]:
time_sql_3 / time_pandas_3

0.951055943970572

Zatem czas utworzenia ramki danych za pomocą pakietu "pandas" jest również zbliżony do czasu utworzenia ramki danych za pomocą zapytania SQL.

### Zadanie 4

#### a) Zapytanie SQL

In [655]:
wyn_sql_4 = pd.read_sql_query("""
SELECT DISTINCT
    Users.Id,
    Users.DisplayName,
    Users.Reputation,
    Users.Age,
    Users.Location
FROM (
        SELECT
            Name, UserID
        FROM Badges
        WHERE Name IN (
            SELECT
                Name
             FROM Badges
             WHERE Class = 1
              GROUP BY Name
              HAVING COUNT(*) BETWEEN 2 AND 10
        )
        AND Class = 1
    ) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId = Users.Id
""", conn)

In [656]:
wyn_sql_4.head(10)

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273,27.0,"London, UK"
1,19,VMAtm,18556,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360,NaN,Down underer
4,793,mindcorrosive,10531,32.0,Bulgaria
5,693,RoflcoptrException,33300,NaN,None
6,6669,Relaxed,69405,NaN,None
7,1737,Gayot Fow,70237,NaN,"London, United Kingdom"
8,39065,Pont,1004,NaN,Austria
9,19400,phoog,34342,50.0,"New York, NY"


#### b) Rozwiązanie z pakietu "pandas"

In [657]:
Zbior_z_Name = Badges.loc[Badges.Class == 1]

In [658]:
Zbior_z_Name = Zbior_z_Name.groupby("Name").apply(len).reset_index().rename(columns = {0: "Ile_Name"})
Zbior_z_Name = Zbior_z_Name.loc[(Zbior_z_Name.Ile_Name >= 2) & (Zbior_z_Name.Ile_Name <= 10)]
Zbior_z_Name = pd.DataFrame(Zbior_z_Name.loc[:, "Name"])

In [659]:
ValuableBadges = Badges[Badges.Name.isin(Zbior_z_Name.Name)]
ValuableBadges = ValuableBadges.loc[ValuableBadges.Class == 1]
ValuableBadges = ValuableBadges.loc[:, ["Name", "UserId"]]
ValuableBadges.head(10)

,Name,UserId
15158,Constable,108
15159,Constable,19
15160,Constable,101
19643,Reversal,466
22472,Sheriff,108
22473,Sheriff,793
22474,Sheriff,693
49097,visas,6669
52321,visas,1737
97413,Sheriff,101


#### Interpretacja zapytania dla ramki ValuableBadges

W ramce danych ValuableBadges wyświetlamy dla każdej nazwy odznaki ID użytkownika. Rozpatrujemy nazwy, gdzie ich ilość jest większa, bądź równa 2 oraz mniejsza, bądź równa 10 oraz nazwy są z klasy 1.

In [660]:
zlacz_ValuableBadges_Users = pd.merge(ValuableBadges, Users, left_on = "UserId", right_on = "Id", how = "inner")

In [661]:
wyn_pandas_4 = zlacz_ValuableBadges_Users.loc[:, ["UserId", "DisplayName", "Reputation", "Age", "Location"]].rename(columns = {"UserId":"Id"}).drop_duplicates().reset_index(drop = True)
wyn_pandas_4.head(10)

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273,27.0,"London, UK"
1,19,VMAtm,18556,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360,NaN,Down underer
4,793,mindcorrosive,10531,32.0,Bulgaria
5,693,RoflcoptrException,33300,NaN,NaN
6,6669,Relaxed,69405,NaN,NaN
7,1737,Gayot Fow,70237,NaN,"London, United Kingdom"
8,39065,Pont,1004,NaN,Austria
9,19400,phoog,34342,50.0,"New York, NY"


#### Interpretacja zapytania dla ramki wynikowej

W otrzymanej ramce danych wyświetlamy ID użytkownika, wyświetlaną nazwę użytkownika, jego reputację podaną w postaci liczbowej, wiek oraz lokalizację użytkownika. Bierzemy użytkownika takiego, że ma odznakę z 1 klasy oraz oznak o danej nazwie jest więcej, bądź równo 2 oraz mniej, bądź równo 10.

#### Porównanie wyników

In [662]:
wyn_sql_4.equals(wyn_pandas_4)

True

Stąd oba wyniki są tożsame.

#### Porównanie czasów obu rozwiązań.

##### a) Zapytanie SQL

In [663]:
rozw_sql_4 = '''
def fun_sql_4():
    wyn_sql_4 = pd.read_sql_query("""
    SELECT DISTINCT
        Users.Id,
        Users.DisplayName,
        Users.Reputation,
        Users.Age,
        Users.Location
    FROM (
            SELECT
                Name, UserID
            FROM Badges
            WHERE Name IN (
                SELECT
                    Name
                 FROM Badges
                 WHERE Class = 1
                  GROUP BY Name
                  HAVING COUNT(*) BETWEEN 2 AND 10
            )
            AND Class = 1
        ) AS ValuableBadges
    JOIN Users ON ValuableBadges.UserId = Users.Id
    """, conn)
'''

In [664]:
time_sql_4 = 0
for i in range(100):
    time_sql_4 = time_sql_4 + ti.timeit(stmt = rozw_sql_4)

time_sql_4 = time_sql_4 / 100

time_sql_4

0.06347191500000918

In [665]:
f"Czyli średni czas utworzenia ramki danych za pomocą zapytania SQL, to {time_sql_4} sekund."

'Czyli średni czas utworzenia ramki danych za pomocą zapytania SQL, to 0.06347191500000918 sekund.'

##### b) Rozwiązanie z pakietu "pandas"

In [666]:
rozw_pandas_4 = '''
def fun_pandas_4():
    Zbior_z_Name = Badges.loc[Badges.Class == 1]
    Zbior_z_Name = Zbior_z_Name.groupby("Name").apply(len).reset_index().rename(columns = {0: "Ile_Name"})
    Zbior_z_Name = Zbior_z_Name.loc[(Zbior_z_Name.Ile_Name >= 2) & (Zbior_z_Name.Ile_Name <= 10)]
    Zbior_z_Name = pd.DataFrame(Zbior_z_Name.loc[:, "Name"])
    ValuableBadges = Badges[Badges.Name.isin(Zbior_z_Name.Name)]
    ValuableBadges = ValuableBadges.loc[ValuableBadges.Class == 1]
    ValuableBadges = ValuableBadges.loc[:, ["Name", "UserId"]]
    zlacz_ValuableBadges_Users = pd.merge(ValuableBadges, Users, left_on = "UserId", right_on = "Id", how = "inner")
    wyn_pandas_4 = zlacz_ValuableBadges_Users.loc[:, ["UserId", "DisplayName", "Reputation", "Age", "Location"]].rename(columns = {"UserId":"Id"}).drop_duplicates().reset_index(drop = True)
'''

In [667]:
time_pandas_4 = 0
for i in range(100):
    time_pandas_4 = time_pandas_4 + ti.timeit(stmt = rozw_pandas_4)

time_pandas_4 = time_pandas_4 / 100

time_pandas_4

0.064088916000037

In [668]:
f"Czyli średni czas utworzenia ramki danych za pomocą pakietu 'pandas', to {time_pandas_4} sekund."

"Czyli średni czas utworzenia ramki danych za pomocą pakietu 'pandas', to 0.064088916000037 sekund."

In [669]:
time_pandas_4 / time_sql_4

1.0097208505530002

Zatem czas utworzenia ramki danych za pomocą pakietu "pandas" jest także zbliżony do czasu utworzenia ramki danych za pomocą zapytania SQL.

### Zadanie 5

#### a) Zapytanie SQL

In [670]:
wyn_sql_5 = pd.read_sql_query("""
SELECT
    Questions.Id,
    Questions.Title,
    BestAnswers.MaxScore,
    Posts.Score AS AcceptedScore,
    BestAnswers.MaxScore - Posts.Score AS Difference
FROM (
        SELECT Id, ParentId, MAX(Score) AS MaxScore
        FROM Posts
        WHERE PostTypeId == 2
        GROUP BY ParentId
    ) AS BestAnswers
JOIN (
        SELECT * FROM Posts
        WHERE PostTypeId == 1
    ) AS Questions
    ON Questions.Id = BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId = Posts.Id
WHERE Difference > 50
ORDER BY Difference DESC
""", conn)

In [671]:
wyn_sql_5.head(10)

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


#### b) Rozwiązanie z pakietu "pandas"

In [672]:
Posts_z_TypeId_2 = Posts.loc[Posts.PostTypeId == 2]
Czesc_bez_Id = Posts_z_TypeId_2.groupby("ParentId")["Score"].agg(max).reset_index().rename(columns = {"Score":"MaxScore"})
BestAnswers = Czesc_bez_Id.merge(Posts_z_TypeId_2, how = "inner", right_on = ["ParentId", "Score"], \
left_on = ["ParentId", "MaxScore"])[["Id", "ParentId", "MaxScore"]]       

BestAnswers.head(10)

,Id,ParentId,MaxScore
0,393,1.0,7
1,2911,2.0,19
2,56,4.0,8
3,770,5.0,13
4,12,6.0,49
5,471,8.0,11
6,178,9.0,20
7,17,10.0,107
8,198,11.0,24
9,1608,13.0,7


#### Interpretacja zapytania dla ramki BestAnswers

Ramka danych BestAnswers wyświetla, jaki każdy ID użytkownika o danym ID rodzica, ma najlepszy wynik. Rozpatrujemy posty o identyfikatorze typu postu równym 2.

In [673]:
Questions = Posts.loc[Posts.PostTypeId == 1]

#### Interpretacja zapytania dla ramki Questions

Ramka danych Questions wyświetla wszystkie dostępne informacje o postach, dla których identyfikator typu danego postu jest równy 1.

In [674]:
Zlacz_BestAnswers_Questions = pd.merge(BestAnswers, Questions, how = "inner", left_on = "ParentId", right_on = "Id")[["Id_y", "Title", "MaxScore", "AcceptedAnswerId"]].rename(columns = {"Id_y":"Id"})

Zlacz_Posts_z_tamtym = pd.merge(Posts, Zlacz_BestAnswers_Questions, how = "inner", left_on = "Id", right_on = "AcceptedAnswerId")

Zlacz_Posts_z_tamtym = Zlacz_Posts_z_tamtym.rename(columns = {"Id_y":"Id", "Title_y":"Title", "Score":"AcceptedScore"})

Zlacz_Posts_z_tamtym["Difference"] = Zlacz_Posts_z_tamtym.MaxScore - Zlacz_Posts_z_tamtym.AcceptedScore

Zlacz_Posts_z_tamtym = Zlacz_Posts_z_tamtym.loc[Zlacz_Posts_z_tamtym.Difference > 50]

Zlacz_Posts_z_tamtym = Zlacz_Posts_z_tamtym.sort_values(by = "Difference", ascending = False)

Zlacz_Posts_z_tamtym = Zlacz_Posts_z_tamtym.rename(columns = {"Score":"AcceptedScore"})

Zlacz_Posts_z_tamtym = Zlacz_Posts_z_tamtym.rename(columns = {"Title_y":"Title"})

wyn_pandas_5 = Zlacz_Posts_z_tamtym.loc[:, ["Id", "Title", "MaxScore", "AcceptedScore", "Difference"]].reset_index(drop = True)

wyn_pandas_5.head(10)

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


#### Interpretacja zapytania dla ramki wynikowej

W otrzymanej ramce danych wyświetlamy dla każdego tytułu o danym ID: najlepszy uzyskany wynik, wynik zaakceptowany oraz różnicę między wynikiem najlepszym, a zaakceptowanym.

#### Porównanie wyników

In [675]:
wyn_sql_5.equals(wyn_pandas_5)

True

Stąd oba wyniki są tożsame.

#### Porównanie czasów obu rozwiązań.

##### a) Zapytanie SQL

In [676]:
rozw_sql_5 = '''
def fun_sql_5():
    wyn_sql_5 = pd.read_sql_query("""
    SELECT
        Questions.Id,
        Questions.Title,
        BestAnswers.MaxScore,
        Posts.Score AS AcceptedScore,
        BestAnswers.MaxScore - Posts.Score AS Difference
    FROM (
            SELECT Id, ParentId, MAX(Score) AS MaxScore
            FROM Posts
            WHERE PostTypeId == 2
            GROUP BY ParentId
        ) AS BestAnswers
    JOIN (
            SELECT * FROM Posts
            WHERE PostTypeId == 1
        ) AS Questions
        ON Questions.Id = BestAnswers.ParentId
    JOIN Posts ON Questions.AcceptedAnswerId = Posts.Id
    WHERE Difference > 50
    ORDER BY Difference DESC
    """, conn)
'''

In [677]:
time_sql_5 = 0
for i in range(100):
    time_sql_5 = time_sql_5 + ti.timeit(stmt = rozw_sql_5)

time_sql_5 = time_sql_5 / 100

time_sql_5

0.06602390800002468

In [678]:
f"Czyli średni czas utworzenia ramki danych za pomocą zapytania SQL, to {time_sql_5} sekund."

'Czyli średni czas utworzenia ramki danych za pomocą zapytania SQL, to 0.06602390800002468 sekund.'

##### b) Rozwiązanie z pakietu "pandas"

In [679]:
rozw_pandas_5 = '''
def fun_pandas_5():
    Posts_z_TypeId_2 = Posts.loc[Posts.PostTypeId == 2]
    Czesc_bez_Id = Posts_z_TypeId_2.groupby("ParentId")["Score"].agg(max).reset_index().rename(columns = {"Score":"MaxScore"})
    BestAnswers = Czesc_bez_Id.merge(Posts_z_TypeId_2, how = "inner", right_on = ["ParentId", "Score"], \
    left_on = ["ParentId", "MaxScore"])[["Id", "ParentId", "MaxScore"]]     
    Questions = Posts.loc[Posts.PostTypeId == 1]
    Zlacz_BestAnswers_Questions = pd.merge(BestAnswers, Questions, how = "inner", left_on = "ParentId", right_on = "Id")[["Id_y", "Title", "MaxScore", "AcceptedAnswerId"]].rename(columns = {"Id_y":"Id"})
    Zlacz_Posts_z_tamtym = pd.merge(Posts, Zlacz_BestAnswers_Questions, how = "inner", left_on = "Id", right_on = "AcceptedAnswerId")
    Zlacz_Posts_z_tamtym = Zlacz_Posts_z_tamtym.rename(columns = {"Id_y":"Id", "Title_y":"Title", "Score":"AcceptedScore"})
    Zlacz_Posts_z_tamtym["Difference"] = Zlacz_Posts_z_tamtym.MaxScore - Zlacz_Posts_z_tamtym.AcceptedScore
    Zlacz_Posts_z_tamtym = Zlacz_Posts_z_tamtym.loc[Zlacz_Posts_z_tamtym.Difference > 50]
    Zlacz_Posts_z_tamtym = Zlacz_Posts_z_tamtym.sort_values(by = "Difference", ascending = False)
    Zlacz_Posts_z_tamtym = Zlacz_Posts_z_tamtym.rename(columns = {"Score":"AcceptedScore"})
    Zlacz_Posts_z_tamtym = Zlacz_Posts_z_tamtym.rename(columns = {"Title_y":"Title"})
    wyn_pandas_5 = Zlacz_Posts_z_tamtym.loc[:, ["Id", "Title", "MaxScore", "AcceptedScore", "Difference"]].reset_index(drop = True)
'''

In [680]:
time_pandas_5 = 0
for i in range(100):
    time_pandas_5 = time_pandas_5 + ti.timeit(stmt = rozw_pandas_5)

time_pandas_5 = time_pandas_5 / 100

time_pandas_5

0.08064109200022358

In [681]:
f"Czyli średni czas utworzenia ramki danych za pomocą pakietu 'pandas', to {time_pandas_5} sekund."

"Czyli średni czas utworzenia ramki danych za pomocą pakietu 'pandas', to 0.08064109200022358 sekund."

In [682]:
time_sql_5 / time_pandas_5

0.8187377720510236

Zatem czas utworzenia ramki danych za pomocą pakietu "pandas" jest również zbliżony do czasu utworzenia ramki danych za pomocą zapytania SQL.

#### Podsumowanie odnośnie czasu tworzenia ramki danych za pomocą zapytania SQL w porównaniu do tworzenia ramki za pomocą pakietu "pandas"

Można zauważyć więc, że ogólnie, dla obu metod, czasy wykonania ramki danych, są zbliżone.

Po skończonej pracy zamykam połączenie następującą komendą:

In [683]:
conn.close()